In [8]:
import numpy as np

In [78]:
#QUBIT base info
#pure states for now, mixed state add later
#multiparticle states can come later too.


class qubit:
    #Base qubit: has internal (private) methods for executing arbitrary rotations. Density matrix is the truest register for internal state.
    #density matrix goes as: first column [0][0] [0][1], second column [1][0] [1][1]
        #so each list in the wider array is a column
    rho = np.array([[1,0],[0,0]], dtype = complex)

    #paulis. px = pauli x, etc.
    i2 = np.array([[1,0],[0,1]])
    px = np.array([[0,1],[1,0]])
    py = np.array([[0,1j],[-1j,0]])
    pz = np.array([[1,0],[0,-1]])
    # avoid using pVec. it is treated as a (1, 3, 2, 2) tensor instead of a vector
    pVec = np.array([[px,py,pz]])
    
    #methods wishlist:
        # arbitrary rotation, pauli gates, get bloch vector.
    
    
    # initialize in z down state
    def __init__(self):
        self.rho = [[1+0j,0+0j],[0+0j,0+0j]]

    # set density state to one reflecting the given z basis [[up, down]] input
    # \ket{state}\bra{state}, outer product
    def setState(self, zBasisVec):
        self.rho = np.outer(zBasisVec, zBasisVec)
    
    #set density state to blcoh vector equivalent
    #pauli decomposition
    def setStateVector(self, blochVec):
        self.rho = 0.5*(self.i2 + blochVec[0][0]*self.px + blochVec[0][1]*self.py + blochVec[0][2]*self.pz)

    # set density state to matrix input
    def setStateDensity(self, matrix):
        self.rho = matrix

    # return density state matrix
    def out(self):
        return( np.around(self.rho ) , 5)
    # normalize trace to one and return dmatrix copy
    def outNormalized(self):
        a = np.trace(self.rho)
        rhoNormalized = np.zeros((2,2), dtype = complex)
        
        for i in range(len(self.rho)):
            for j in range(len(self.rho[0])):
                rhoNormalized[i][j] = self.rho[i][j] / a
        return( np.around(rhoNormalized) , 5)

    # pure if trace( rho^2 ) == 1
    #note: python 3.5 "@" operator does matrix multiplication for base arrays, np.matmul does the same, np.dot works fine for 2d but not for higher dimension
    def isPure(self):
        return (1 == np.around( np.trace(np.dot(self.rho, self.rho))) , 5)
        
    # expected value of a hermitian operator "measurement" on rho: Tr(rho * operator)
    def expectedValue(self, operator):
        return np.around( np.trace(np.dot(self.rho, operator)) , 5)


    def blochVector(self):
        b = np.zeros((1, 3), dtype = complex)
        b[0][0] = self.expectedValue(self.px)
        b[0][1] = self.expectedValue(self.py)
        b[0][2] = self.expectedValue(self.pz)
        return np.around(b, 5)

    # rotate around 3d axis (unit) vector by theta. 
    # normalize nVector, then construct rotation matrix from paulis: U_{n, theta} = I\cos(\theta) + i\sin(\theta) * (n \cdot \Vec{\sigma})
    # multiply rot^\dag * rho * rot
    # wikipedia Bloch_sphere > rotation operators 
    def rotate(self, nVec, theta): #theta in radians, nVector must be in the format [[n_x, n_y, n_z]] with two brackets (so n by 1 matrix)
        nVec /= np.linalg.norm(nVec)
        rot = np.array(self.i2) * np.cos(0.5*theta) - (0+1j) * np.sin(0.5*theta) * ( nVec[0][0]*self.px + nVec[0][1]*self.py + nVec[0][2]*self.pz )
        self.rho = np.dot( np.dot( np.conjugate( np.transpose(rot) ), self.rho), rot)
        self.rho = np.around(self.rho, 5)



newq = qubit()


print("initial bloch vec: ", newq.blochVector())

newq.rotate([[1,0,0]], np.pi/2)

print("\n new bloch vec: ", newq.blochVector())




initial bloch vec:  [[0.+0.j 0.+0.j 1.+0.j]]

 new bloch vec:  [[ 0.+0.j -1.+0.j  0.+0.j]]


In [ ]:
# debug sequence

